In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset
file_path = 'bat_charge(4).csv'  # Update this to your file path
data = pd.read_csv(file_path)

In [3]:
# Convert the 'Time' column to datetime format
data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y %H:%M')

In [4]:
data

,Time,VDC,Isp
0,2022-05-07 22:30:00,363.616943,10
1,2022-05-07 22:31:00,365.859985,10
2,2022-05-07 22:31:00,366.348267,10
3,2022-05-07 22:31:00,366.683960,10
4,2022-05-07 22:31:00,366.943359,10
...,...,...,...
2875,2022-06-07 06:33:00,360.046387,10
2876,2022-06-07 06:33:00,360.031128,10
2877,2022-06-07 06:33:00,360.031128,10
2878,2022-06-07 06:33:00,360.015869,10


In [5]:
# Set 'Time' as the index
data.set_index('Time', inplace=True)


In [6]:
# Resample the data if needed (assuming 1-minute intervals)
data = data.resample('T').mean().interpolate()  # Resampling per minute and interpolating missing values


C:\Users\0931\AppData\Local\Temp\ipykernel_14580\3366766124.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data = data.resample('T').mean().interpolate()  # Resampling per minute and interpolating missing values


In [7]:
data

,VDC,Isp
Time,,
2022-05-07 22:30:00,363.616943,10.0
2022-05-07 22:31:00,366.727193,10.0
2022-05-07 22:32:00,367.904663,10.0
2022-05-07 22:33:00,368.682862,10.0
2022-05-07 22:34:00,369.328817,10.0
...,...,...
2022-06-07 06:29:00,360.115051,10.0
2022-06-07 06:30:00,360.084534,10.0
2022-06-07 06:31:00,360.074361,10.0


In [8]:
# Normalize the 'VDC' column
scaler = MinMaxScaler(feature_range=(0, 1))
data['VDC_normalized'] = scaler.fit_transform(data['VDC'].values.reshape(-1, 1))


In [9]:
# Prepare the data for LSTM
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps, 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)


In [10]:
# Use the normalized VDC for modeling
sequence_data = data['VDC_normalized'].values.reshape(-1, 1)
n_steps = 30  # Number of time steps to look back

In [11]:
# Create sequences
X, y = create_sequences(sequence_data, n_steps)

# Reshape X for LSTM [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

C:\Users\0931\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping], verbose=1)


Epoch 1/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.0344 - val_loss: 1.3318e-06
Epoch 2/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 8.7360e-07 - val_loss: 2.6665e-06
Epoch 3/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.5547e-06 - val_loss: 1.0129e-06
Epoch 4/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.1867e-06 - val_loss: 1.8884e-06
Epoch 5/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 2.0175e-06 - val_loss: 3.1581e-06
Epoch 6/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 5.7077e-06 - val_loss: 4.7226e-07
Epoch 7/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 2.1617e-06 - val_loss: 2.0205e-06
Epoch 8/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 2.1167e-05 - val_loss: 2.5108e-06
Epoch 9/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 1.0819e-06 - val_loss: 2.9969e-07
Epoch 10/100
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 6.1884e-06 - val_loss: 3.1423e-07
Epoch 11/100
1092/1092 ━━━━━━━━

In [14]:
# Save the model
model.save('lstm_vdc_model.h5')

In [15]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=1)


273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.2099e-07


In [16]:

# Predict on the test set
y_pred = model.predict(X_test)

273/273 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [17]:
# Reverse normalization to get actual VDC values
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred)

In [18]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test_actual, y_pred_actual)
mae = mean_absolute_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R^2: {r2}')


MSE: 2.2404363970319708e-05
MAE: 0.0026843348914590754
R^2: 0.9998667004762997


In [19]:





# Predict the next 90 minutes
last_sequence = data['VDC_normalized'].values[-n_steps:]
predictions = []

for _ in range(90):
    next_pred = model.predict(last_sequence.reshape(1, n_steps, 1))
    predictions.append(next_pred[0, 0])
    last_sequence = np.append(last_sequence[1:], next_pred)

# Reverse normalization to get actual VDC predictions
predictions_actual = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

print(predictions_actual)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━

In [20]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import MeanSquaredError

# Load the saved model
model = load_model('lstm_vdc_model.h5', compile=False)
model.compile(optimizer='adam', loss=MeanSquaredError())

# Load the dataset to get the last sequence of data
file_path = 'bat_charge(4).csv'
data = pd.read_csv(file_path)

# Convert the 'Time' column to datetime format
data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y %H:%M')

# Set 'Time' as the index
data.set_index('Time', inplace=True)

# Resample the data if needed (assuming 1-minute intervals)
data = data.resample('T').mean().interpolate()  # Resampling per minute and interpolating missing values

# Normalize the 'VDC' column
scaler = MinMaxScaler(feature_range=(0, 1))
data['VDC_normalized'] = scaler.fit_transform(data['VDC'].values.reshape(-1, 1))

# Prepare the last sequence of data
n_steps = 30  # Number of time steps to look back
last_sequence = data['VDC_normalized'].values[-n_steps:]

# Function to predict VDC for the next n minutes
def predict_future_vdc(minutes):
    predictions = []
    sequence = last_sequence.copy()
    
    for _ in range(minutes):
        next_pred = model.predict(sequence.reshape(1, n_steps, 1))
        predictions.append(next_pred[0, 0])
        sequence = np.append(sequence[1:], next_pred)
    
    # Reverse normalization to get actual VDC predictions
    predictions_actual = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    
    return predictions_actual

# User input for the number of minutes
minutes = int(input("Enter the number of minutes to predict the VDC for: "))

# Make the prediction
predicted_values = predict_future_vdc(minutes)

# Display the predicted values
print(f"Predicted VDC values for the next {minutes} minutes:")
print(predicted_values)


C:\Users\0931\AppData\Local\Temp\ipykernel_14580\816796782.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data = data.resample('T').mean().interpolate()  # Resampling per minute and interpolating missing values


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted VDC values for the next 23 minutes:
[[360.07632]
 [360.06653]
 [360.05743]
 [360.04907]
 [360.0414 ]
 [360.03424]
 

# DNN

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [3]:
# Load the dataset
file_path = 'bat_charge(4).csv'  # Update this to your file path
data = pd.read_csv(file_path)
data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y %H:%M')


In [4]:
# Set 'Time' as the index
data.set_index('Time', inplace=True)
# Resample the data if needed (assuming 1-minute intervals)
data = data.resample('T').mean().interpolate()  # Resampling per minute and interpolating missing values

# Normalize the 'VDC' column
scaler = MinMaxScaler(feature_range=(0, 1))
data['VDC_normalized'] = scaler.fit_transform(data['VDC'].values.reshape(-1, 1))


In [5]:
# Prepare the data for DNN
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps, 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)


In [6]:
# Use the normalized VDC for modeling
sequence_data = data['VDC_normalized'].values.reshape(-1, 1)
n_steps = 30  # Number of time steps to look back
# Create sequences
X, y = create_sequences(sequence_data, n_steps)


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
# Define the DNN model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=n_steps))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping], verbose=1)


Epoch 1/100
1092/1092 [==============================] - 2s 1ms/step - loss: 0.0018 - val_loss: 1.4456e-06
Epoch 2/100
1092/1092 [==============================] - 1s 1ms/step - loss: 9.1323e-07 - val_loss: 9.0515e-07
Epoch 3/100
1092/1092 [==============================] - 1s 1ms/step - loss: 8.7182e-07 - val_loss: 7.8745e-07
Epoch 4/100
1092/1092 [==============================] - 1s 1ms/step - loss: 9.8369e-07 - val_loss: 1.6482e-06
Epoch 5/100
1092/1092 [==============================] - 1s 1ms/step - loss: 1.2372e-06 - val_loss: 1.5776e-06
Epoch 6/100
1092/1092 [==============================] - 1s 1ms/step - loss: 2.1774e-06 - val_loss: 7.4005e-07
Epoch 7/100
1092/1092 [==============================] - 1s 1ms/step - loss: 2.2847e-06 - val_loss: 1.1369e-05
Epoch 8/100
1092/1092 [==============================] - 1s 981us/step - loss: 8.6072e-06 - val_loss: 6.5726e-06
Epoch 9/100
1092/1092 [==============================] - 1s 1ms/step - loss: 4.6092e-06 - val_loss: 1.5039e-06
Epo

In [9]:
# Save the model
model.save('dnn_vdc_model.h5')

C:\Users\0871\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=1)


273/273 [==============================] - 0s 691us/step - loss: 7.4005e-07


In [11]:
# Predict on the test set
y_pred = model.predict(X_test)
# Reverse normalization to get actual VDC values
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_actual = scaler.inverse_transform(y_pred)

273/273 [==============================] - 0s 853us/step


In [12]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test_actual, y_pred_actual)
mae = mean_absolute_error(y_test_actual, y_pred_actual)
r2 = r2_score(y_test_actual, y_pred_actual)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R^2: {r2}')


MSE: 0.00011431875252605193
MAE: 0.007846188713017552
R^2: 0.9993198362925221


In [13]:

# Load the saved model
from tensorflow.keras.models import load_model

model = load_model('dnn_vdc_model.h5')

# Load the dataset to get the last sequence of data
data = pd.read_csv(file_path)
data['Time'] = pd.to_datetime(data['Time'], format='%d-%m-%Y %H:%M')
data.set_index('Time', inplace=True)
data = data.resample('T').mean().interpolate()  # Resampling per minute and interpolating missing values

# Normalize the 'VDC' column
scaler = MinMaxScaler(feature_range=(0, 1))
data['VDC_normalized'] = scaler.fit_transform(data['VDC'].values.reshape(-1, 1))

# Prepare the last sequence of data
last_sequence = data['VDC_normalized'].values[-n_steps:]

# Function to predict VDC for the next n minutes
def predict_future_vdc(minutes):
    predictions = []
    sequence = last_sequence.copy()
    
    for _ in range(minutes):
        next_pred = model.predict(sequence.reshape(1, n_steps))
        predictions.append(next_pred[0, 0])
        sequence = np.append(sequence[1:], next_pred)
    
    # Reverse normalization to get actual VDC predictions
    predictions_actual = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    
    return predictions_actual

# User input for the number of minutes
minutes = int(input("Enter the number of minutes to predict the VDC for: "))

# Make the prediction
predicted_values = predict_future_vdc(minutes)

# Display the predicted values
print(f"Predicted VDC values for the next {minutes} minutes:")
print(predicted_values)


1/1 [==============================] - 0s 22ms/step
Predicted VDC values for the next 10 minutes:
[[360.07254]
 [360.06406]
 [360.0637 ]
 [360.0554 ]
 [360.0332 ]
 [360.02548]
 [360.01883]
 [360.02216]
 [360.01657]
 [360.01828]]
